In [1]:
from biollm.task.perturbation.pert_task import PertTask


config_file = '../../config/pert/scmamba_pert.toml'
obj = PertTask(config_file)
pert_data, gene_ids = obj.make_dataset()

/home/share/huadjyin/home/s_qiuping1/app/miniconda3/envs/scgpt/lib/python3.9/site-packages/lightning/fabric/plugins/environments/xla.py:18: DeprecationWarning: `ModuleAvailableCache` is a special case of `RequirementCache`. Please use `RequirementCache(module=...)` instead.
  from lightning.fabric.accelerators.tpu import _XLA_AVAILABLE, TPUAccelerator
/home/share/huadjyin/home/s_qiuping1/app/miniconda3/envs/scgpt/lib/python3.9/site-packages/lightning/fabric/__init__.py:36: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning.fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__("pkg_resources").declare_namespace(__name__)
/home/share/huadjyin/home/s_qiuping1/app/miniconda3/envs/scgpt/lib/python3.9/site-packages/pkg_resources/__init__.py:2348: DeprecationWarning: Deprecated call 

Using simple batchnorm instead of domain specific batchnorm


Found local copy...


before filter: (63585, 4399)
match 1.0 genes in vocab of size 4399
after filter: View of AnnData object with n_obs × n_vars = 63585 × 4399
    obs: 'condition', 'cell_type', 'dose_val', 'control', 'condition_name', 'pert_gene_in_vocab'
    var: 'gene_name', 'is_in_vocab'
    uns: 'non_dropout_gene_idx', 'non_zeros_gene_idx', 'rank_genes_groups_cov_all', 'top_non_dropout_de_20', 'top_non_zero_de_20'


Found local copy...
Local copy of pyg dataset is detected. Loading...
Done!
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['TIMM23+ctrl' 'AMIGO3+ctrl' 'KCTD16+ctrl']
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:18
Done!
Creating dataloaders....
Done!


here1


2024-04-18 18:51:22-pert_task[line-67]-INFO: match 4399/4399 genes in vocabulary of size 60697.


In [2]:
best_model_path = obj.args.save_dir + '/best_model.pt'
best_model = obj.load_obj.load_pretrain_model(best_model_path, obj.model)

In [48]:
import torch
from torch.utils.data import DataLoader
from biollm.model.perturbation import ScgptPerturbation
from typing import Dict, Optional
import numpy as np
from anndata import AnnData
import pickle
from biollm.repo.scmamba.scmamba.mambaLM import MambaModel

def eval_scmamba_perturb(
    loader: DataLoader, model: MambaModel, device: torch.device, include_zero_gene: str
, gene_ids: np.ndarray) -> Dict:
    """
    Run model in inference mode using a given data loader
    """
    model.eval()
    model.to(device)
    pert_cat = []
    pred = []
    truth = []
    pred_de = []
    truth_de = []
    results = {}

    print('start to predict: ', len(loader.dataset))
    for itr, batch in enumerate(loader):
        batch.to(device)
        pert_cat.extend(batch.pert)
        sorted_index = batch.sorted_index
        sorted_index = np.argsort(np.array(sorted_index), axis=1)
        with torch.no_grad():
            p = model.pred_perturb(
                batch,
                include_zero_gene=include_zero_gene,
                gene_ids=gene_ids,
            ).detach().cpu().numpy().astype(float)
            t = batch.y.detach().cpu().numpy().astype(float)
            # Differentially expressed genes
            for itr, de_idx in enumerate(batch.de_idx):
                pred_de.append(p[itr, de_idx])
                truth_de.append(t[itr, de_idx])
            pred.extend(p[np.arange(p.shape[0])[:, None], sorted_index])
            truth.extend(t[np.arange(t.shape[0])[:, None], sorted_index])
            
            
    print('end')
    # all genes
    results["pert_cat"] = np.array(pert_cat)
    pred = np.stack(pred)
    truth = np.stack(truth)
    results["pred"] = pred
    results["truth"] = truth
    # results['de_idx'] = de_idxs
    pred_de = np.stack(pred_de)
    truth_de = np.stack(truth_de)
    results["pred_de"] = pred_de
    results["truth_de"] = truth_de
    return results

In [49]:
test_loader = pert_data.dataloader["test_loader"]
test_res = eval_scmamba_perturb(test_loader, best_model, 'cuda:2', 'all', gene_ids)

start to predict:  8695
end


In [50]:
test_res

{'pert_cat': array(['ZNF326+ctrl', 'ZNF326+ctrl', 'ZNF326+ctrl', ..., 'CHERP+ctrl',
        'CHERP+ctrl', 'CHERP+ctrl'], dtype='<U13'),
 'pred': array([[0.00795746, 0.0055275 , 0.00562668, ..., 0.00954437, 0.08312988,
         0.07458496],
        [0.00795746, 0.0055275 , 0.00562668, ..., 0.00839996, 0.08123779,
         0.54931641],
        [0.00795746, 0.0055275 , 0.00562668, ..., 0.00804138, 0.0824585 ,
         0.46166992],
        ...,
        [0.00795746, 0.0055275 , 0.00562668, ..., 0.00978088, 0.08435059,
         0.63378906],
        [0.00795746, 0.0055275 , 0.00562668, ..., 0.00848389, 0.0769043 ,
         0.2578125 ],
        [0.00795746, 0.0055275 , 0.00562668, ..., 0.00813293, 0.23242188,
         0.21643066]]),
 'truth': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.98109186,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.

In [51]:
from biollm.repo.gears.inference import evaluate, compute_metrics

test_metrics, test_pert_res = compute_metrics(test_res)

In [11]:
test_metrics

{'mse': 0.010639110014391306,
 'mse_de': 0.182111431037015,
 'pearson': 0.9865111277927033,
 'pearson_de': 0.9534139080623021}

In [ ]:
test_metrics

In [52]:
from biollm.evaluate.bm_metrices_pert import compute_perturbation_metrics

test_metrics = compute_perturbation_metrics(
    test_res, pert_data.adata[pert_data.adata.obs["condition"] == "ctrl"]
)
print(test_metrics)

{'pearson': 0.9865111277927006, 'pearson_de': 0.9534139080623021, 'pearson_delta': 0.34561967989313486, 'pearson_de_delta': 0.7731520751238459}


In [53]:
from biollm.repo.gears.inference import deeper_analysis, non_dropout_analysis
import numpy as np

deeper_res = deeper_analysis(pert_data.adata, test_res)
non_dropout_res = non_dropout_analysis(pert_data.adata, test_res)

metrics = ["pearson_delta", "pearson_delta_de"]
metrics_non_dropout = [
    "pearson_delta_top20_de_non_dropout",
    "pearson_top20_de_non_dropout",
]
subgroup_analysis = {}
for name in pert_data.subgroup["test_subgroup"].keys():
    subgroup_analysis[name] = {}
    for m in metrics:
        subgroup_analysis[name][m] = []

    for m in metrics_non_dropout:
        subgroup_analysis[name][m] = []

for name, pert_list in pert_data.subgroup["test_subgroup"].items():
    for pert in pert_list:
        for m in metrics:
            subgroup_analysis[name][m].append(deeper_res[pert][m])

        for m in metrics_non_dropout:
            subgroup_analysis[name][m].append(non_dropout_res[pert][m])

for name, result in subgroup_analysis.items():
    for m in result.keys():
        mean_value = np.mean(subgroup_analysis[name][m])
        print("test_" + name + "_" + m + ": " + str(mean_value))


test_combo_seen0_pearson_delta: nan
test_combo_seen0_pearson_delta_de: nan
test_combo_seen0_pearson_delta_top20_de_non_dropout: nan
test_combo_seen0_pearson_top20_de_non_dropout: nan
test_combo_seen1_pearson_delta: nan
test_combo_seen1_pearson_delta_de: nan
test_combo_seen1_pearson_delta_top20_de_non_dropout: nan
test_combo_seen1_pearson_top20_de_non_dropout: nan
test_combo_seen2_pearson_delta: nan
test_combo_seen2_pearson_delta_de: nan
test_combo_seen2_pearson_delta_top20_de_non_dropout: nan
test_combo_seen2_pearson_top20_de_non_dropout: nan
test_unseen_single_pearson_delta: 0.3456196798931348
test_unseen_single_pearson_delta_de: 0.773152075123846
test_unseen_single_pearson_delta_top20_de_non_dropout: 0.7728769223680013
test_unseen_single_pearson_top20_de_non_dropout: 0.9508552704076959
